In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image, ImageTk
from faker import Faker
fake = Faker()

from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

In [ ]:
review_text = "I love this product.I hate it. I love going skiing. It is so good. I would recommend it for every smartwatch enthusiast."
keywords = kw_model.extract_keywords(review_text, 
                                     keyphrase_ngram_range=(1,1) ,
                                     stop_words='english', 
                                     highlight=True,
                                     top_n=6)

keywords_list= list(dict(keywords).keys())

print(keywords_list)

# Basic Dataset erstellen

3 Kategorien: Hotels, Restaurants & Aktivitäten

In [ ]:
df_rest = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
df_act = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
df_hotel = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')

print('Restaurants (Shape): ', df_rest.shape)
print('Activities (Shape): ', df_act.shape)
print('Hotels (Shape): ', df_hotel.shape)

In [ ]:
# print in a looop the columns names and print the different values for each column

for i in df_rest.columns:
    print(i)
    print(df_rest[i].unique())


In [ ]:
columns_to_keep = [
    "stars",
    "publishedAtDate",
    "name",
    "text",
    "title",
    "subTitle",
    "description",
    "price",
    "totalScore",
    "likesCount",
    "isLocalGuide",
    "reviewId",
    "categoryName",
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9",
    "reviewUrl",
    "reviewerId",
    "reviewerNumberOfReviews",
    "reviewerPhotoUrl",
    "reviewerUrl",
    "reviewsCount",
    "scrapedAt",
    "state",
    "street",
    "temporarilyClosed",
    "url",
    "website",

    # at least for restaurants
    "categories/0",
    "categories/1",
    "categories/2",
    "categoryName"
]

In [ ]:
df_rest = df_rest.dropna(subset=["reviewImageUrls/0"])
df_rest = df_rest[columns_to_keep]

# add a column with genre and for evey row the value restaurant

df_rest['genre'] = 'restaurant'
print(df_rest.shape)

In [ ]:
df_act = df_act.dropna(subset=["reviewImageUrls/0"])
df_act = df_act[columns_to_keep]
df_act['genre'] = 'activity'
print(df_act.shape)

In [ ]:
df_hotel = df_hotel.dropna(subset=["reviewImageUrls/0"])
df_hotel = df_hotel[columns_to_keep]
df_hotel['genre'] = 'hotel'
print(df_hotel.shape)

In [ ]:
df = pd.concat([df_rest, df_act, df_hotel], ignore_index=True)
print(df.shape)

In [ ]:
image_urls_cols = [
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9"
]

for col in image_urls_cols:
    df[col + "_hr"] = df[col]
    df[col + "_hr"] = df[col + "_hr"].str.replace("=w150-h150-k-no-p", "=w256-h256-p-k-no")

print(df.shape)

# for index, row in df.iterrows():
#     print("row: ", index)
#     print(row["reviewImageUrls/0"])
#     print(row["reviewImageUrls/0_hr"])

In [ ]:
df['text'] = df['text'].astype(str)
print("Convertion done.")

df['keywords'] = df['text'].apply(lambda x: kw_model.extract_keywords(x,keyphrase_ngram_range=(1,1) ,
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=4))

print(df_keywords_test['keywords'])

df.to_csv('01_Data\merged_with_keywords.csv', index=False)


In [ ]:
print(df_keywords_test['keywords'])

#only store the keywords in a list not the scores

df_keywords_test['keywords_only'] = df_keywords_test['keywords'].apply(lambda x: [i[0] for i in x])
print(df_keywords_test['keywords_only'])

In [ ]:
# print the keywords for the first 5 rows

print(df['keywords'].head())

In [ ]:
for index, row in df.iterrows():
    print("row: ", index)
    print(row["description"])

## Gradio Anwendung Versuch

### HTML Bewertung im Stil von Google Maps erstellen

In [ ]:
def generate_review_html(stars, date, user, is_local_guide, text, user_image, review_images):
    full_stars = int(stars)
    half_star = (stars % 1) > 0
    empty_stars = 5 - full_stars - half_star
    full_star_html = '<span class="star full">&#9733;</span>'
    half_star_html = '<span class="star half">&#9733;</span>'
    empty_star_html = '<span class="star empty">&#9733;</span>'
    stars_html = (full_star_html * full_stars) + (half_star_html * half_star) + (empty_star_html * empty_stars)
    user_name = user if not is_local_guide else f"{user} (local guide)"
    
    html = f"""
    <style>
      .star.full {{
        color: #f1c40f;
      }}
      .star.half {{
        color: #f1c40f;
        position: relative;
      }}
      .star.half::before {{
        content: '\\2605';
        position: absolute;
        left: 0;
        color: #ddd;
      }}
      .star.empty {{
        color: #ddd;
      }}
    </style>
    <div class="review">
      <div class="user">
        <img src={user_image} alt="User profile picture">
        <div class="user-info">
          <h4>{user_name}</h4>
          <p>Submitted on {date}</p>
        </div>
      </div>
      <div class="rating">
        {stars_html}
      </div>
      <div class="review-text">
        <p>{text}</p>
      </div>
      <div class="review-image">
        <img src={review_images} alt="Restaurant image">
      </div>
    </div>
    """
    return html

In [ ]:
def generate_deepfake(review_about, stars):
    text = fake.text()
    stars = fake.random_int(min=1, max=5, step=1)
    date = fake.date_between(start_date='-1y', end_date='today')
    user = fake.name()
    local_guide = fake.boolean(chance_of_getting_true=7)
    html_review = generate_review_html(stars, date, user, local_guide, text, user_image="https://via.placeholder.com/50/%22%20alt=/%22Restaurant%20image/", review_images = "https://lh5.googleusercontent.com/p/AF1QipPsYMpfUpwmH5ltkPNIeDbgPivvBstyJCea32vJ=w150-h150-k-no-p")
    return html_review

def verify_review(stars, date, user, local_guide, text, image):
    return "FAKE", "0.9"


In [ ]:
with gr.Blocks() as demo:
    with gr.Tab ("Create Fake Review"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        generate_btn = gr.Button("Generate Multi-Modal Review")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake, inputs=[review_about, review_stars], outputs=[html_output])
    
    with gr.Tab ("Verify Review"):
        with gr.Row(): 
            input_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
            input_date = gr.Textbox(label="Date", default="2023-01-01")
            input_user = gr.Textbox(label="User")
            input_local_guide = gr.Checkbox(label="Local Guide")
       
        input_text = gr.Textbox(label="Review Text")
        input_image = gr.Image(label="Review Image")
        verify_btn = gr.Button("Verify Review")
        output_text = gr.Textbox(label="Generated Text")
        output_score = gr.Textbox(label="Score")
        verify_btn.click(fn=verify_review, inputs=[input_stars, input_date, input_user, input_local_guide, input_text, input_image], outputs=[output_text, output_score])

demo.launch(debug=True)